# Importar Librerias

In [5]:
# Para dar inicio al laboratorio se procede a importar librerias a utilizar
import pandas as pd
import numpy as np
from pycaret.datasets import get_data
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from pycaret.regression import *
import warnings
from sklearn.ensemble import IsolationForest

# Cargar Datos

In [6]:
file_path = 'train.csv'
data = pd.read_csv(file_path)

# División del Data Set 80% training y 20% test

In [7]:
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# EDA y Profiling del dataset utilizando PyCaret

In [ ]:
exp1 = setup(data=train_data, 
             target='SalePrice', 
             session_id=2023, 
             normalize=True, 
             normalize_method='minmax', 
             transformation=True)

data=  get_data(dataset='train')

# Descripción de las alertas generadas en el EDA

In [ ]:
data = get_data(dataset='train',  profile = True )

# Ingenieria de Caracteristicas

In [11]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest


# Dividir el conjunto de datos en variables predictoras (X) y variable objetivo (y) si es necesario
# X = data.drop('variable_objetivo', axis=1)
# y = data['variable_objetivo']

# Definir las columnas numéricas y categóricas
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

# Construir preprocesador
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Puedes ajustar la estrategia de imputación según tus necesidades
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Puedes ajustar la estrategia de imputación según tus necesidades
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Eliminación de outliers usando Isolation Forest
outlier_detector = IsolationForest(contamination=0.05)  # Ajusta la contaminación según sea necesario
outliers = outlier_detector.fit_predict(data[numeric_features])

# Filtrar filas sin outliers
data = data[outliers == 1]


# Aplicar preprocesador
X_preprocessed = preprocessor.fit_transform(data)


# Entrenamiento y selcción de Modelos Automaticamente

In [12]:
pip install scikit-learn matplotlib


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Suponiendo que ya tienes X_preprocessed y y definidos después del preprocesamiento anterior

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Definir el clasificador RandomForest
rf_classifier = RandomForestClassifier(random_state=42)

# Definir la cuadrícula de hiperparámetros para buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Realizar la búsqueda de cuadrícula en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluar el modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

# Imprimir resultados
print("Mejores hiperparámetros:", best_params)
print("Exactitud del modelo:", accuracy)
print("Informe de clasificación:\n", classification_report_result)

# Mostrar la importancia de las características
feature_importances = best_rf_model.feature_importances_

# Crear un gráfico de barras para mostrar la importancia de las características
feature_names = list(data.columns)  # Asegúrate de que el orden de las características coincida
plt.barh(range(len(feature_importances)), feature_importances, align='center')
plt.yticks(range(len(feature_importances)), feature_names)
plt.xlabel('Importancia de características')
plt.ylabel('Características')
plt.title('Importancia de características para el modelo RandomForest')
plt.show()

# Optimización de Hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Definir el clasificador (por ejemplo, RandomForest)
classifier = RandomForestClassifier()

# Definir la cuadrícula de hiperparámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Realizar la búsqueda de cuadrícula en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

# Imprimir resultados
print("Mejores hiperparámetros:", best_params)
print("Exactitud del modelo:", accuracy)
print("Informe de clasificación:\n", classification_report_result)
